# Group project - August 27, 2024
Group Members -
1. Nivan
2. Pavithra
3. Yoonjeong
4. Shishir 

In [2]:
# Upgrade the Snowflake Python connector to the latest version.

!pip install --upgrade snowflake-connector-python

In [3]:
# Install the xmltodict library for parsing XML data in Python

! pip install xmltodict 

In [4]:
# Importing libraries

import snowflake.connector
import xmltodict
from datetime import datetime
import os
import glob
import psycopg2
import csv
import pandas as pd

In [18]:
# Connecting with Snowflake
import snowflake.connector

conn = snowflake.connector.connect(
    user='denggyi0211',
    password='Denggyi960921',
    account='covzexm-pib26437'
    )

In [22]:
# Creating cursor object

cs = conn.cursor()

In [23]:
# Creating Database monthly_po_data

cs.execute("CREATE DATABASE IF NOT EXISTS monthly_po_data")

In [24]:
# Creating schema and stage using monthly_po_data database
cs.execute("USE DATABASE monthly_po_data")
cs.execute("CREATE SCHEMA IF NOT EXISTS public")
cs.execute("CREATE STAGE IF NOT EXISTS my_stage")

In [25]:
# Creating code that we could run everytime we open the file
cs.execute("USE DATABASE monthly_po_data")
cs.execute("USE SCHEMA public")

# Question 1

In [27]:
# Creating table for Purchases
cs.execute("""
CREATE OR REPLACE TABLE purchases (
   PurchaseOrderID INT,
   SupplierID INT,
   OrderDate DATE,
   DeliveryMethodID INT,
   ContactPersonID INT,
   ExpectedDeliveryDate DATE,
   SupplierReference STRING,
   IsOrderFinalized BOOLEAN,
   Comments STRING,
   InternalComments STRING,
   LastEditedBy INT,
   LastEditedWhen VARCHAR,
   PurchaseOrderLineID INT,
   StockItemID INT,
   OrderedOuters INT,
   Description STRING,
   ReceivedOuters INT,
   PackageTypeID INT,
   ExpectedUnitPricePerOuter FLOAT,
   LastReceiptDate DATE NULL,
   IsOrderLineFinalized BOOLEAN,
   Right_LastEditedBy INT,
   Right_LastEditedWhen STRING
)
""")


print("Table created successfully.")


Table created successfully.


In [28]:
# Specify the directory path and find CSV files
directory_path = "/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data"
csv_file = glob.glob(os.path.join(directory_path, "*.csv"))
print(csv_file)

['/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2019-1.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2020-5.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2020-4.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2022-4.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2019-2.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2020-6.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2020-7.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2019-3.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Monthly PO Data/2022-5.csv', '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop0

In [29]:
# Iterate through the CSV files and execute the PUT command
for files in csv_file:
   put_command = f"PUT 'file://{files}' @my_stage AUTO_COMPRESS=TRUE"
   cs.execute(put_command)
   

In [30]:
# Load the data from the stage into the purchases table
query = """
COPY INTO purchases
FROM @my_stage
FILE_FORMAT = (
   TYPE = 'CSV',
   FIELD_OPTIONALLY_ENCLOSED_BY = '"',
   NULL_IF = ('NULL', '')
   SKIP_HEADER = 1,
   TIMESTAMP_FORMAT = 'MM/DD/YYYY HH:MI'
)
ON_ERROR = 'CONTINUE';
"""
cs.execute(query)

In [31]:
# Modifying Purchase Table
cs.execute("ALTER TABLE purchases DROP COLUMN Comments")
cs.execute("ALTER TABLE purchases DROP COLUMN InternalComments")
cs.execute("ALTER TABLE purchases ADD COLUMN IF NOT EXISTS POAmount FLOAT")

# Question 2

In [32]:
# Calculating the POAmount for each purchase order

cs.execute("""
   UPDATE purchases
   SET POAmount = (SELECT SUM(ReceivedOuters * ExpectedUnitPricePerOuter)
          FROM purchases AS P2
          WHERE PURCHASES.PurchaseOrderID = P2.PurchaseOrderID)
""")


# Question 3

In [67]:
# Checking if the data is in valid form

def is_valid_date(date_str):
   if isinstance(date_str, str):
       try:
           datetime.strptime(date_str, '%Y-%m-%d')
           return True
       except ValueError:
           return False
   return False 


In [68]:
# Parse the XML File

with open('/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/Supplier_Transaction/Supplier_Transactions_XML.xml', 'r') as file:
   xml_content = file.read()
   data_dict = xmltodict.parse(xml_content)
print("XML file parsed successfully.")

XML file parsed successfully.


In [70]:
# Prepare the Data for Snowflake
data_to_insert = []
invalid_rows = []


# Check if the required date fields are valid before adding the row
for row in data_dict['root']['row']:
   if is_valid_date(row['TransactionDate']): 
       data_to_insert.append((
           row['SupplierTransactionID'],
           row['SupplierID'],
           row['TransactionTypeID'],
           row['PurchaseOrderID'],
           row['PaymentMethodID'],
           row['SupplierInvoiceNumber'],
           row['TransactionDate'],
           row['AmountExcludingTax'],
           row['TaxAmount'],
           row['TransactionAmount'],
           row['OutstandingBalance'],
           row.get('FinalizationDate'), 
           row['IsFinalized'],
           row['LastEditedBy'],
           row['LastEditedWhen']
       ))
   else:
       invalid_rows.append(row)


# Print all invalid rows after processing
print(f"Data prepared successfully. Total valid records to insert: {len(data_to_insert)}")
print(f"Total invalid rows: {len(invalid_rows)}")
for invalid_row in invalid_rows:
   print(f"Invalid row: {invalid_row}")

Data prepared successfully. Total valid records to insert: 2438
Total invalid rows: 0


In [71]:
# Creating Table in Snowflake

cs.execute("""
CREATE OR REPLACE TABLE SupplierTransactions (
   SupplierTransactionID NUMBER,
   SupplierID NUMBER,
   TransactionTypeID NUMBER,
   PurchaseOrderID NUMBER,
   PaymentMethodID NUMBER,
   SupplierInvoiceNumber STRING,
   TransactionDate DATE,
   AmountExcludingTax NUMBER,
   TaxAmount NUMBER,
   TransactionAmount NUMBER,
   OutstandingBalance NUMBER,
   FinalizationDate DATE,
   IsFinalized BOOLEAN,
   LastEditedBy NUMBER,
   LastEditedWhen TIMESTAMP
);
""")
print("Table created successfully.")


Table created successfully.


In [72]:
# Inserting Data into Snowflake

insert_query = """
INSERT INTO SupplierTransactions (
   SupplierTransactionID, SupplierID, TransactionTypeID, PurchaseOrderID,
   PaymentMethodID, SupplierInvoiceNumber, TransactionDate, AmountExcludingTax,
   TaxAmount, TransactionAmount, OutstandingBalance, FinalizationDate,
   IsFinalized, LastEditedBy, LastEditedWhen
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cs.executemany(insert_query, data_to_insert)
print("Data inserted successfully.")

Data inserted successfully.


In [73]:
# Commit the current transaction to save changes to the database and confirm successful completion.

conn.commit()
print("Transaction committed successfully.")

Transaction committed successfully.


In [74]:
# Verify the data by counting the number of records in the SupplierTransactions table after insertion.

cs.execute("SELECT COUNT(*) FROM SupplierTransactions")
print(f"Number of records inserted: {cs.fetchone()[0]}")

Number of records inserted: 2438


# Question4

In [41]:
# Query to select all finalized orders by joining purchases and 
# SupplierTransactions where both orders and transactions are finalized.

cs.execute("""
   SELECT
   *
FROM
   purchases AS p
INNER JOIN
   SupplierTransactions AS st
ON
   p.PurchaseOrderID = st.PurchaseOrderID
   AND p.SupplierID = st.SupplierID
WHERE
   p.IsOrderFinalized = TRUE
   AND st.IsFinalized = TRUE;
""")


# Fetching all results
results = cs.fetchall()

# Printing out the results
for row in results:
   print(row)

(510, 4, datetime.date(2019, 11, 1), 7, 2, datetime.date(2019, 11, 21), '293092', True, 4, '11/4/2019 7:00', 1981, 77, 703, '"The Gu" red shirt XML tag t-shirt (White) XXS', 703, 6, 84.0, datetime.date(2019, 11, 4), True, 4, '11/4/2019 7:00', 361470.0, 75552, 4, 5, 510, 4, '551', datetime.date(2019, 11, 4), 361470, 54221, 415691, 0, datetime.date(2019, 11, 4), True, 8, datetime.datetime(2019, 11, 4, 9, 0))
(510, 4, datetime.date(2019, 11, 1), 7, 2, datetime.date(2019, 11, 21), '293092', True, 4, '11/4/2019 7:00', 1982, 78, 924, '"The Gu" red shirt XML tag t-shirt (White) XS', 924, 6, 84.0, datetime.date(2019, 11, 4), True, 4, '11/4/2019 7:00', 361470.0, 75552, 4, 5, 510, 4, '551', datetime.date(2019, 11, 4), 361470, 54221, 415691, 0, datetime.date(2019, 11, 4), True, 8, datetime.datetime(2019, 11, 4, 9, 0))
(510, 4, datetime.date(2019, 11, 1), 7, 2, datetime.date(2019, 11, 21), '293092', True, 4, '11/4/2019 7:00', 1983, 80, 347, '"The Gu" red shirt XML tag t-shirt (White) M', 347, 6, 8

# Question 5

In [42]:
#This query creates or replaces the purchase_orders_and_invoices table by joining purchases 
# with SupplierTransactions, adding a new column invoiced_vs_quoted that calculates the difference 
# between the invoiced amount and the quoted amount.

cs.execute("""
CREATE OR REPLACE TABLE purchase_orders_and_invoices AS
SELECT
   p.*,
   st.AmountExcludingTax - p.POAmount AS invoiced_vs_quoted
FROM
   purchases AS p
INNER JOIN
   SupplierTransactions AS st
ON
   p.PurchaseOrderID = st.PurchaseOrderID
WHERE
   p.SupplierID = st.SupplierID;
""")

# Fetching all results
results = cs.fetchall()

# Printing out the results
for row in results:
   print(row)

('Table PURCHASE_ORDERS_AND_INVOICES successfully created.',)


# Question 6

In [43]:
# Function to extract data from the supplier_case table in a PostgreSQL database and save it as a CSV file.

def extract_supplier_case_to_csv(pg_conn_params, output_csv_path):
   try:
       # Connecting to the PostgreSQL database
       conn = psycopg2.connect(**pg_conn_params)
       cursor = conn.cursor()
  
       # Define the SQL query to extract data
       sql_query = "COPY (SELECT * FROM supplier_case) TO STDOUT WITH CSV HEADER"


       # Write the data to a CSV file
       with open(output_csv_path, 'w') as f_output:
           cursor.copy_expert(sql_query, f_output)
     
       print(f"Data successfully extracted to {output_csv_path}")
 
   except Exception as e:
       print(f"Error: {e}")


pg_conn_params = {
   'dbname': 'postgres',
   'user': 'jovyan',
   'password': 'postgres',
   'host': '127.0.0.1',
   'port': '8765'
}


output_csv_path = 'supplier_case_data.csv'
extract_supplier_case_to_csv(pg_conn_params, output_csv_path)

Data successfully extracted to supplier_case_data.csv


In [44]:
# Function to upload a CSV file from the local file system to a specified Snowflake stage.

def upload_csv_to_snowflake_stage(cs, local_csv_path, stage_name):
# Uploading the CSV file
   command = f"PUT 'file://{local_csv_path}' @{stage_name} AUTO_COMPRESS=TRUE"
   cs.execute(command)
   print(f"{local_csv_path} successfully uploaded to {stage_name}")


upload_csv_to_snowflake_stage(cs, '/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/supplier_case_data.csv', 'my_stage')

/home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/supplier_case_data.csv successfully uploaded to my_stage


In [45]:
# Function to generate a SQL CREATE TABLE statement based on the 
# column names and data types inferred from the first row of a CSV file.

def generate_create_table_statement(csv_file_path, table_name):
   df = pd.read_csv(csv_file_path, nrows=1) 
 
   columns = []
    # Iterate over each column in the DataFrame to determine its SQL data type
   for column in df.columns:
       dtype = pd.api.types.infer_dtype(df[column])
       if dtype == 'integer':
           sql_type = 'INTEGER'
       elif dtype == 'floating':
           sql_type = 'FLOAT'
       elif dtype == 'datetime':
           sql_type = 'TIMESTAMP'
       else:
           sql_type = 'STRING'
     
       columns.append(f"{column} {sql_type}")
 
   create_table_sql = f"CREATE OR REPLACE TABLE {table_name} (\n" + ",\n".join(columns) + "\n);"


   return create_table_sql



table_name = 'supplier_case'
create_table_sql = generate_create_table_statement('supplier_case_data.csv', table_name)
print(create_table_sql)

CREATE OR REPLACE TABLE supplier_case (
supplierid INTEGER,
suppliername STRING,
suppliercategoryid INTEGER,
primarycontactpersonid INTEGER,
alternatecontactpersonid INTEGER,
deliverymethodid INTEGER,
postalcityid INTEGER,
supplierreference STRING,
bankaccountname STRING,
bankaccountbranch STRING,
bankaccountcode INTEGER,
bankaccountnumber INTEGER,
bankinternationalcode INTEGER,
paymentdays INTEGER,
internalcomments FLOAT,
phonenumber STRING,
faxnumber STRING,
websiteurl STRING,
deliveryaddressline1 STRING,
deliveryaddressline2 STRING,
deliverypostalcode INTEGER,
deliverylocation STRING,
postaladdressline1 STRING,
postaladdressline2 STRING,
postalpostalcode INTEGER,
lasteditedby INTEGER,
validfrom STRING,
validto STRING
);


In [46]:
# Creating table in Snowflake
def create_table_in_snowflake(cs, create_table_sql):
   cs.execute(create_table_sql)
   print("Table created successfully in Snowflake.")


# Loading data from stage to Snowflake
def load_data_from_stage(cs, table_name, stage_name, file_name):
   load_command = f"""
   COPY INTO {table_name}
   FROM @{stage_name}/{file_name}
   FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY = '\"', SKIP_HEADER = 1)
   ON_ERROR = 'CONTINUE'
   """
   cs.execute(load_command)
   print("Data loaded into Snowflake table successfully.")


create_table_in_snowflake(cs, create_table_sql)
load_data_from_stage(cs, 'supplier_case', 'my_stage', 'supplier_case_data.csv.gz')

Table created successfully in Snowflake.
Data loaded into Snowflake table successfully.


# Question 7

In [49]:
# Create a table to store the zip code data
cs.execute("""
    CREATE OR REPLACE TABLE zip_code_mapping (
    GEOID STRING,
    ALAND FLOAT,
    AWATER FLOAT,
    ALAND_SQMI FLOAT,
    AWATER_SQMI FLOAT,
    INTPTLAT FLOAT,
    INTPTLONG FLOAT
    )
""")


print("Table created successfully.")

Table created successfully.


In [52]:
# Uploading a file to a Snowflake stage 
result = cs.execute("PUT 'file:///home/jovyan/git/rsm-mgta403b-case3-2024-new-rsm-yop018/MGTA464_SQLProject/2021_Gaz_zcta_national.txt' @MY_STAGE AUTO_COMPRESS=FALSE OVERWRITE=TRUE")

for row in result:
   print(f"File: {row[0]}")
   print(f"Status: {row[6]}")

File: 2021_Gaz_zcta_national.txt
Status: UPLOADED


In [54]:
# Defining the SQL query to copy data from the stage into the table zip_code_mapping

copy_into_query = """
COPY INTO zip_code_mapping
FROM @my_stage
FILE_FORMAT = (
    TYPE = 'CSV',
    FIELD_DELIMITER = '\t',  
    SKIP_HEADER = 1
)
ON_ERROR = 'CONTINUE';
"""
cs.execute(copy_into_query)

print("Data successfully copied from the stage into the zip_code_mapping table.")

Data successfully copied from the stage into the zip_code_mapping table.


In [55]:
# Creating View (distance_station)

distance_station_query = """
CREATE OR REPLACE VIEW distance_station AS
SELECT a.GEOID as zip_code, b.NOAA_WEATHER_STATION_ID as station_id,
    ST_DISTANCE(
        ST_MAKEPOINT(a.INTPTLONG, a.INTPTLAT),
        ST_MAKEPOINT(b.LONGITUDE, b.LATITUDE)
    ) AS distance,
    ROW_NUMBER() OVER (
        PARTITION BY a.GEOID
        ORDER BY ST_DISTANCE(ST_MAKEPOINT(a.INTPTLONG, a.INTPTLAT), ST_MAKEPOINT(b.LONGITUDE, b.LATITUDE))
        ) AS "rank"
FROM monthly_po_data.public.zip_code_mapping a
JOIN WEATHER__ENVIRONMENT.cybersyn.NOAA_WEATHER_STATION_INDEX b
ON a.GEOID = b.zip_name
"""


cs.execute(distance_station_query)

print("View distance_station created successfully.")

View distance_station created successfully.


In [58]:
# Creating View (closest_station)

closest_station = """
CREATE OR REPLACE VIEW nearest_station AS
SELECT *
FROM monthly_po_data.public.distance_station
WHERE "rank" = 1
"""

cs.execute(closest_station)

print("View closest_station created successfully.")

View closest_station created successfully.


In [64]:
# Creating View (supplier_zip_code_weather)

supplier_zip_code_weather_query = """
CREATE OR REPLACE VIEW supplier_zip_code_weather AS
SELECT
    a.PostalPostalCode,
    c.date,
    MIN(c.value) AS Temperature
FROM monthly_po_data.public.SUPPLIER_CASE AS a
JOIN nearest_station AS b 
ON a.PostalPostalCode = b.zip_code
JOIN WEATHER__ENVIRONMENT.cybersyn.NOAA_WEATHER_METRICS_TIMESERIES c
ON b.station_id = c.NOAA_WEATHER_STATION_ID
WHERE c.variable_name = 'Maximum Temperature'
GROUP BY a.PostalPostalCode, c.date;
"""

cs.execute("SELECT * FROM supplier_zip_code_weather LIMIT 10")
print(cs.fetchall())
print("View supplier_zip_code_weather created successfully.")

[(42437, datetime.date(2008, 4, 5), Decimal('13.300000')), (42437, datetime.date(2005, 8, 15), Decimal('33.900000')), (42437, datetime.date(2008, 1, 19), Decimal('2.800000')), (42437, datetime.date(2005, 8, 14), Decimal('36.100000')), (42437, datetime.date(2008, 11, 1), Decimal('23.300000')), (42437, datetime.date(2007, 10, 22), Decimal('27.200000')), (42437, datetime.date(2008, 9, 2), Decimal('32.200000')), (42437, datetime.date(2007, 6, 1), Decimal('27.200000')), (42437, datetime.date(2007, 4, 11), Decimal('16.700000')), (42437, datetime.date(2008, 1, 17), Decimal('2.800000'))]
View supplier_zip_code_weather created successfully.


# Question 8

In [66]:
# Creating Final_supplier_weather_data Table

final_supplier_weather_data_query = ("""
   CREATE OR REPLACE TABLE final_supplier_weather_data AS
    SELECT             
        sc.suppliername,     
        szw.temperature  
    FROM supplier_case sc  
    JOIN supplier_zip_code_weather szw
    ON sc.PostalPostalCode = szw.PostalPostalCode 
    WHERE szw.temperature IS NOT NULL  
""")

cs.execute(final_supplier_weather_data_query)

print("final_supplier_weather_data table created successfully.")

final_supplier_weather_data table created successfully.
